In [1]:
# Peter Karras 2020
'''Takes GPP and date information from old master file and creates a new Machine Learning Master File
   With one column representing file names for the converted CSVs and the other column being their respective
   GPPs as pulled from the old master file or calculated with linear interpolation'''

'Takes GPP and date information from old master file and creates a new Machine Learning Master File\n   With one column representing file names for the converted CSVs and the other column being their respective\n   GPPs as pulled from the old master file or calculated with linear interpolation'

In [2]:
import pandas as pd
import os
import time

In [3]:
# Takes in a string that represents name of csv file and 
# converts that string to one that matches the date string present
# in the old master file (old master file refers to the raw excel file
# filled with extraneous data from BCI)that contains the date/times 
# for recorded GPP values
def change_csv_date(csv):
    year = csv[2:4]
    month = csv[5:7]
    day = csv[8:10]
    hour = csv[11:13]
    minute = csv[14:16]
    second = "00"
    if(int(hour) == 0):
        hour = "0"
    elif(int(hour) < 10):
        hour = hour[1:2]
    # No leading zeroes in day or month
    if(int(month) < 10):
        month = month[1:2]
    if(int(day) < 10):
        day = day[1:2]
    result = month + "/" + day + "/" + year + " " + hour + ":" + minute #+ ":" + second + " " + timeDay
    return result

In [4]:
# Takes in csv date (in format 'YYYY_MM_DD_HH_MM_bci_t') and checks to see if minutes are divisble by div
def check_csv_date(csv, div):
    result = False
    minute = int(csv[14:16])
    remainder = minute % div
    if(remainder == 0):
        result = True
    else:
        result = False
    return result

In [5]:
print(check_csv_date("2015_03_14_14_00_bci_t.csv",30))

True


In [6]:
# Grabs all GPP values and dates from master file (as defined above)
masterfileLocation = r"C:\Users\Peter\Desktop\Thermo_R_Code\FinalThermo\new2"
os.chdir(masterfileLocation)
col_list = ["date", "GPP"]
masterFileName = "all-30min-Apr2018_working.csv"
dateGPPList = pd.read_csv(masterFileName, usecols=col_list) 
dateGPPList = dateGPPList.set_index("date")



csvdataLocation = r"C:\Users\Peter\Desktop\Thermo_R_Code\FinalThermo\new2\csvdata9000"
(_, _, csvfiles) = next(os.walk(csvdataLocation))
print(csvfiles[0])

2015_08_25_07_00_bci_t.csv


In [7]:
# This chunk of code matches the file names from the old master file to the
# names gotten from the "csvdata" folder, so that each thermal csv file has it's
# corresponding GPP value matched to it
# NOTE: In addition to matching GPP values, this code also fills in GPP values
# for those images without recorded values through linear interpoaltion
# NOTE: Currently only works when first CSV is from a 30 min increment
# NOTE: Anomolies in data make it so sometimes data moves in 1 minute increments, 
# currently this function cannot circumvents this and these files must be removed

# NOTE: Still has issues when there is a gap within masterfile data, currently just files
# intermeditate values with last known val for GPP

dictList = list()

counter = 0
numLoops = len(csvfiles)
oldGPP = -1.1
for i in range(0, numLoops, 6):
    #GPPDataDict.clear()
    currFile = csvfiles[i]
    adaptedDate = change_csv_date(currFile)
    try:
        currGPP = dateGPPList.at[adaptedDate,"GPP"]
    except:
        print("Gap in time: ", adaptedDate)
    if(i == 0):
        #GPPDataDict.update({currFile : currGPP})
        GPPDataDict = dict()
        GPPDataDict.update({"GPP" : currGPP})
        GPPDataDict.update({"FILE" : currFile})
        dictList.append(GPPDataDict)
        oldGPP = currGPP
        continue
        
    GPPDataDictTemp = dict()
    GPPDataDictTemp.update({"GPP" : currGPP})
    GPPDataDictTemp.update({"FILE" : currFile})
    
    
    diffGPP = currGPP - oldGPP
    increment = diffGPP / 6.0
    for j in range(i-5, i, 1):
        oldGPP += increment
        currFile = csvfiles[j]
        #print("Here", currFile)
        GPPDataDict = dict()
        GPPDataDict.update({"GPP" : oldGPP})
        GPPDataDict.update({"FILE" : currFile})
        dictList.append(GPPDataDict)
    dictList.append(GPPDataDictTemp)
    oldGPP = currGPP + increment
    oldGPP = currGPP

Gap in time:  9/11/15 20:30
Gap in time:  9/11/15 21:00
Gap in time:  9/11/15 21:30
Gap in time:  9/17/15 11:45
Gap in time:  9/17/15 12:15
Gap in time:  9/17/15 12:45
Gap in time:  9/17/15 13:15
Gap in time:  9/17/15 13:45
Gap in time:  9/17/15 14:15
Gap in time:  9/17/15 14:45
Gap in time:  9/17/15 15:15
Gap in time:  9/17/15 15:45
Gap in time:  9/17/15 16:15
Gap in time:  9/17/15 16:45
Gap in time:  9/17/15 17:15
Gap in time:  9/17/15 17:45
Gap in time:  9/17/15 18:15
Gap in time:  9/17/15 18:45
Gap in time:  9/17/15 19:15
Gap in time:  9/17/15 19:45
Gap in time:  9/17/15 20:15
Gap in time:  9/17/15 20:45
Gap in time:  9/17/15 21:15
Gap in time:  9/17/15 21:45
Gap in time:  9/17/15 22:15
Gap in time:  9/17/15 22:45
Gap in time:  9/17/15 23:15
Gap in time:  9/17/15 23:45
Gap in time:  9/18/15 0:15
Gap in time:  9/18/15 0:45
Gap in time:  9/18/15 1:15
Gap in time:  9/18/15 1:45
Gap in time:  9/18/15 2:15
Gap in time:  9/18/15 2:45
Gap in time:  9/18/15 3:15
Gap in time:  9/18/15 3:45


In [8]:
# This writes a NEW master file, with one column containing the
# thermal csv file names, and the other column containing their
# GPP values as determined from previous code. New file name is 
# determined by variable csv_file1

import csv
saveLocation = csvdataLocation
os.chdir(saveLocation)
csv_file1 = "populatedGPPs.csv"
csv_columns = ['FILE','GPP']
try:
    with open(csv_file1, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, delimiter=',', lineterminator='\n', fieldnames=csv_columns)
        writer.writeheader()
        for data in dictList:
            writer.writerow(data)
except IOError:
    print("I/O error")
    
